In [1]:
import geopandas as gpd
import pandas as pd
import boto3
from sqlalchemy import create_engine, text
from sqlalchemy.ext.declarative import declarative_base
from config import settings

In [2]:
#BUCKET_NAME = 'ds-interview-sandbox'
#client = boto3.client('s3')
#s3 = boto3.resource('s3')
#bucket = s3.Bucket(BUCKET_NAME)

In [3]:
gdf = gpd.read_file('./uga_regions.geojson')
gdf.columns = [x.lower() for x in gdf.columns]

In [4]:
gdf.head()

,name,iso_code,iso2_code,area_type,geometry
0,Eastern,UG-E,UGA,Regions,"MULTIPOLYGON (((33.51939 -1.00000, 33.52066 -0..."
1,Western,UG-W,UGA,Regions,"MULTIPOLYGON (((29.65173 -0.64149, 29.65195 -0..."
2,Central,UG-C,UGA,Regions,"MULTIPOLYGON (((32.97509 0.90589, 32.97594 0.9..."
3,Northern,UG-N,UGA,Regions,"MULTIPOLYGON (((32.19980 3.50716, 32.20084 3.5..."


In [5]:
# Renaming Cols
cols = {"v001": "cluster_id",
        "v002": "household_no",
        "v003": "respondent_line",
        "v012": "respondent_age",
        "v133": "education_ages",
        "v535": "married",
        "adult_radio_regular": "adult_radio_regular",
        "v751": "HIV_aware",
        "sex": "sex", }
# Read DHS
dhs = pd.read_csv("./uga_dhs_2016.csv")
dhs = dhs.rename(columns=cols)

In [6]:
dhs.head()

,cluster_id,household_no,respondent_line,respondent_age,education_ages,married,HIV_aware,adult_radio_regular,sex
0,1,1,2,19,15,0.0,1,0,female
1,1,3,2,25,9,NaN,1,1,female
2,1,4,1,40,5,1.0,1,0,female
3,1,4,2,21,6,NaN,1,0,female
4,1,4,3,16,9,0.0,1,0,female


In [7]:
def boolify(val: int):
    if val > 0:
        return 'yes'
    elif val == 0:
        return 'no'
    else:
        return None

In [8]:
dhs['married'] = dhs.married.apply(lambda x: boolify(x))
dhs['HIV_aware'] = dhs.HIV_aware.apply(lambda x: boolify(x))
dhs['adult_radio_regular'] = dhs.adult_radio_regular.apply(
    lambda x: boolify(x))


In [9]:
dhs.head()

,cluster_id,household_no,respondent_line,respondent_age,education_ages,married,HIV_aware,adult_radio_regular,sex
0,1,1,2,19,15,no,yes,no,female
1,1,3,2,25,9,None,yes,yes,female
2,1,4,1,40,5,yes,yes,no,female
3,1,4,2,21,6,None,yes,no,female
4,1,4,3,16,9,no,yes,no,female


In [10]:
# Read Custers
ug_clusters = pd.read_csv("./ug_clusters.csv")
ug_clusters = ug_clusters.rename(columns={"v001": "cluster_id"})
ug_clusters = gpd.GeoDataFrame(
    ug_clusters, geometry=gpd.points_from_xy(ug_clusters.LONGNUM, ug_clusters.LATNUM))
ug_clusters.columns = [x.lower() for x in ug_clusters.columns]
ug_clusters = ug_clusters.set_crs(4326, allow_override=True)


In [11]:
ug_clusters.head()

,cluster_id,latnum,longnum,geometry
0,1,0.320188,32.568206,POINT (32.56821 0.32019)
1,2,0.340653,32.593627,POINT (32.59363 0.34065)
2,3,0.313103,32.566556,POINT (32.56656 0.31310)
3,4,0.353368,32.558144,POINT (32.55814 0.35337)
4,5,0.367388,32.594357,POINT (32.59436 0.36739)


In [19]:
# Merge Regions and Cluster
clus_reg = ug_clusters.sjoin(gdf, how="left")

In [20]:
clus_reg.head()

,cluster_id,latnum,longnum,geometry,index_right,name,iso_code,iso2_code,area_type
0,1,0.320188,32.568206,POINT (32.56821 0.32019),2.0,Central,UG-C,UGA,Regions
1,2,0.340653,32.593627,POINT (32.59363 0.34065),2.0,Central,UG-C,UGA,Regions
2,3,0.313103,32.566556,POINT (32.56656 0.31310),2.0,Central,UG-C,UGA,Regions
3,4,0.353368,32.558144,POINT (32.55814 0.35337),2.0,Central,UG-C,UGA,Regions
4,5,0.367388,32.594357,POINT (32.59436 0.36739),2.0,Central,UG-C,UGA,Regions


In [23]:
out = dhs.merge(clus_reg, how="left", on="cluster_id")

In [25]:
out.columns

Index(['cluster_id', 'household_no', 'respondent_line', 'respondent_age',
       'education_ages', 'married', 'HIV_aware', 'adult_radio_regular', 'sex',
       'latnum', 'longnum', 'geometry', 'index_right', 'name', 'iso_code',
       'iso2_code', 'area_type'],
      dtype='object')

In [6]:
#s3 = boto3.resource('s3')
#bucket = s3.Bucket('ds-interview-sandbox')
# Iterates through all the objects, doing the pagination for you. Each obj
# is an ObjectSummary, so it doesn't contain the body. You'll need to call
# get to get the whole body.
#for obj in bucket.objects.filter(Prefix='shared/ug'):
    #key = obj.key
    #print(key)

In [7]:
#for obj in bucket.objects.filter(Prefix='shared/ug'):
    #key = obj.key
    #s3.Bucket(BUCKET_NAME).download_file(key, f"./{key.split('/')[-1]}")
    #gdf = gpd.read_file(f"./{key.split('/')[-1]}")
    #gdf.to_postgis()
    #print(gdf.shape)

In [7]:
DATABASE_URL = f'postgresql+psycopg2://{settings.database_username}:{settings.database_password}@{settings.database_host}:{settings.database_port}/{settings.database_name}'
engine = create_engine(DATABASE_URL)

In [21]:
df = pd.read_csv("./ug_clusters.csv")
ug_clusters = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.LONGNUM, df.LATNUM))
ug_clusters.columns = [x.lower() for x in ug_clusters.columns]
ug_clusters = ug_clusters.set_crs(4326, allow_override=True)
ug_clusters.to_postgis("ug_clusters", engine)

In [31]:
df = pd.read_csv("./uga_dhs_2016.csv")
df.columns = [x.lower() for x in df.columns]
df.to_sql("ug_dhs", engine)

842

In [29]:
gdf = gpd.read_file("./uga_regions.geojson")
gdf.columns = [x.lower() for x in gdf.columns]
gdf.to_postgis("ug_regions", engine)

In [12]:
query = text("""SELECT ug_dhs.v751, ug_dhs.v001, ug_clusters.latnum, ug_clusters.longnum, ug_dhs.sex, ug_regions.geometry  
FROM ug_dhs 
JOIN ug_clusters ON ug_dhs.v001 = ug_clusters.v001
JOIN ug_regions ON ug_regions.name = 'Northern'
WHERE ug_dhs.sex = 'male'""")

In [4]:
query = text("select count(dhs.sex) as count from ug_dhs dhs join (select a.v001, (case when ST_contains(b.geometry, a.geometry) then b.name else 'n/a' end) as region FROM public.ug_clusters a, public.ug_regions b) regions on dhs.v001=regions.v001 where regions.region=:region and dhs.sex=:sex and dhs.v751=:aids").bindparams(region='Northern', sex='male', aids=1)


In [88]:
region = "'Northern'"
sex = "'Male'"
aids = 1

In [89]:
#query = "select count(dhs.sex) as count from ug_dhs dhs join (select a.v001, (case when ST_contains(b.geometry, a.geometry) then b.name else 'n/a' end) as region FROM public.ug_clusters a, public.ug_regions b) regions on dhs.v001=regions.v001 where regions.region={region} and dhs.sex={sex} and dhs.v751={aids}"


In [8]:
engine.execute(query).fetchone()[0]


1232